# RAG-based Q&A on D&D #

### As the sentence and later retrieval transformer we had multiple choices to pick from: ###
all-MiniLM-L12-v2 	(Our model is intented to be used as a sentence and short paragraph encoder. Given an input text, it ouptuts a vector which captures the semantic information. The sentence vector may be used for information retrieval, clustering or sentence similarity tasks. - https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2)

multi-qa-distilbert-cos-v1 	("This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and was designed for semantic search. It has been trained on 215M (question, answer) pairs from diverse sources. For an introduction to semantic search, have a look at: SBERT.net - Semantic Search" - https://huggingface.co/sentence-transformers/multi-qa-distilbert-cos-v1)

In [ ]:
%pip install -U datasets huggingface_hub fsspec
%pip install haystack-ai
%pip install google-genai-haystack
%pip install "sentence-transformers>=4.1.0"
%pip install "fsspec==2023.9.2"
%pip install "sentence-transformers>=4.1.0" "huggingface_hub>=0.23.0"
%pip install markdown-it-py mdit_plain pypdf

In [ ]:
import requests
import pprint

url = "https://www.dnd5eapi.co/api/2014"

payload = {}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
answer_whole = response.text
print(answer_whole)

{"ability-scores":"/api/2014/ability-scores","alignments":"/api/2014/alignments","backgrounds":"/api/2014/backgrounds","classes":"/api/2014/classes","conditions":"/api/2014/conditions","damage-types":"/api/2014/damage-types","equipment":"/api/2014/equipment","equipment-categories":"/api/2014/equipment-categories","feats":"/api/2014/feats","features":"/api/2014/features","languages":"/api/2014/languages","magic-items":"/api/2014/magic-items","magic-schools":"/api/2014/magic-schools","monsters":"/api/2014/monsters","proficiencies":"/api/2014/proficiencies","races":"/api/2014/races","rule-sections":"/api/2014/rule-sections","rules":"/api/2014/rules","skills":"/api/2014/skills","spells":"/api/2014/spells","subclasses":"/api/2014/subclasses","subraces":"/api/2014/subraces","traits":"/api/2014/traits","weapon-properties":"/api/2014/weapon-properties"}


In [143]:
url = "https://www.dnd5eapi.co/api/2014/monsters/hybrid-werewolf"
response = requests.request("GET", url, headers=headers, data=payload)
pprint.pprint(response.text)

'Not Found'


In [ ]:

# same as conditions, damage types, magic-schools, rule sections, weapon properties
list_of_indices = ['conditions','damage-types','magic-schools','rule-sections','weapon-properties']

dict_of_conditions = {}
dict_of_damage_types = {}
dict_of_magic_schools = {}
dict_of_rule_sections = {}
dict_of_weapon_properties = {}

def create_dict(type):
    dict_of_response_data = {}
    url = "https://www.dnd5eapi.co/api/2014/"+type
    response = requests.request("GET", url, headers=headers, data=payload)
    resp = response.json()

    for entry in resp['results']:
        response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
        resp2 = response2.json()
    
        response_data = {
            'name': entry['name'],
            'description': "".join(resp2['desc'])
        }
        dict_of_response_data[entry['index']] = response_data
    return dict_of_response_data

dict_of_conditions = create_dict(list_of_indices[0])
dict_of_damage_types = create_dict(list_of_indices[1])
dict_of_magic_schools = create_dict(list_of_indices[2])
dict_of_rule_sections = create_dict(list_of_indices[3])
dict_of_weapon_properties = create_dict(list_of_indices[4])

pprint.pprint(dict_of_conditions)

{'blinded': {'description': "- A blinded creature can't see and automatically "
                            'fails any ability check that requires sight.- '
                            'Attack rolls against the creature have advantage, '
                            "and the creature's attack rolls have "
                            'disadvantage.',
             'name': 'Blinded'},
 'charmed': {'description': "- A charmed creature can't attack the charmer or "
                            'target the charmer with harmful abilities or '
                            'magical effects.- The charmer has advantage on '
                            'any ability check to interact socially with the '
                            'creature.',
             'name': 'Charmed'},
 'deafened': {'description': "- A deafened creature can't hear and "
                             'automatically fails any ability check that '
                             'requires hearing.',
              'name': 'Deafened'},


In [121]:
pprint.pprint(dict_of_weapon_properties)

{'ammunition': {'description': 'You can use a weapon that has the ammunition '
                               'property to make a ranged attack only if you '
                               'have ammunition to fire from the weapon. Each '
                               'time you attack with the weapon, you expend '
                               'one piece of ammunition. Drawing the '
                               'ammunition from a quiver, case, or other '
                               'container is part of the attack (you need a '
                               'free hand to load a one-handed weapon).At the '
                               'end of the battle, you can recover half your '
                               'expended ammunition by taking a minute to '
                               'search the battlefield. If you use a weapon '
                               'that has the ammunition property to make a '
                               'melee attack, you treat the weapon as 

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/traits"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_traits = {}
trait_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    trait_data = {
         'name': name,
          'desc': "".join(resp2["desc"])
    }

    if resp2.get('races'):
         trait_data['races'] = [item['name'] for item in resp2['races']]

    if resp2.get('subraces'):
         trait_data['subraces'] = [item['name'] for item in resp2['subraces']]

    if resp2.get('proficiencies'):
         trait_data['proficiencies'] = [item['name'] for item in resp2['proficiencies']]

    dict_of_traits[entry['index']] = trait_data
pprint.pprint(dict_of_traits)

{'artificers-lore': {'desc': 'Whenever you make an Intelligence (History) '
                             'check related to magic items, alchemical '
                             'objects, or technological devices, you can add '
                             'twice your proficiency bonus, instead of any '
                             'proficiency bonus you normally apply.',
                     'name': "Artificer's Lore",
                     'subraces': ['Rock Gnome']},
 'brave': {'desc': 'You have advantage on saving throw against being '
                   'frightened.',
           'name': 'Brave',
           'races': ['Halfling']},
 'breath-weapon': {'desc': 'You can use your action to exhale destructive '
                           'energy. Your draconic ancestry determines the '
                           'size, shape, and damage type of the '
                           'exhalation.When you use your breath weapon, each '
                           'creature in the area of the exhal

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/rules"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_rules = {}
rule_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('subsections'):
         rule_data['subsection_in_rule_sections'] = [item['name'] for item in resp2['subsections']]

    rule_data = {
        'name' : name,
        'desc': "".join(resp2["desc"])
    }
    dict_of_rules[entry['index']] = rule_data
pprint.pprint(dict_of_rules)

{'adventuring': {'desc': '# Adventuring\n',
                 'name': 'Adventuring',
                 'subsection_in_rule_sections': ['Fantasy-Historical Pantheons',
                                                 'The Planes of Existence']},
 'appendix': {'desc': '# Appendix\n',
              'name': 'Appendix',
              'subsection_in_rule_sections': ['The Order of Combat',
                                              'Movement and Position',
                                              'Actions in Combat',
                                              'Making an Attack',
                                              'Cover',
                                              'Damage and Healing',
                                              'Mounted Combat',
                                              'Underwater Combat']},
 'combat': {'desc': '# Combat\n',
            'name': 'Combat',
            'subsection_in_rule_sections': ['Standard Exchange Rates',
                     

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/spells"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_spells = {}
spell_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('material'):
         spell_data['material'] = resp2['material']

    if resp2.get('subclasses'):
        spell_data['subclasses'] = [item['name'] for item in resp2['subclasses']]

    if resp2.get('higher_level'):
        spell_data['higher_level'] = resp2['higher_level']

    if resp2.get('damage'):
        slot_level_damage = {}
        if 'damage_type' in resp2['damage']:
            spell_data['damage_type'] = [resp2['damage']['damage_type']['name']]
        if 'damage_at_slot_level' in resp2['damage']:
            for slot_level, damage in resp2['damage']['damage_at_slot_level'].items():
                slot_level_damage[slot_level] = damage
            spell_data['damage_at_slot_level'] = slot_level_damage 

    if resp2.get('atack_type'):
       spell_data['attack_type'] = resp2['attack_type']

    spell_data = {
        'name' : name,
        'desc': "".join(resp2["desc"]),
        'range': resp2['range'],
        'components':resp2['components'],
        'ritual':resp2['ritual'],
        'duration': resp2['duration'],
        'concentration': resp2['concentration'],
        'casting_time': resp2['casting_time'],
        'level': resp2['level'],
        'school_of_magic': [resp2['school']['name']],
        'classes': [item['name'] for item in resp2['classes']],
    }
    dict_of_spells[entry['index']] = spell_data
pprint.pprint(dict_of_spells)

{'acid-arrow': {'casting_time': '1 action',
                'classes': ['Wizard'],
                'components': ['V', 'S', 'M'],
                'concentration': False,
                'damage_type': ['Acid'],
                'desc': ['A shimmering green arrow streaks toward a target '
                         'within range and bursts in a spray of acid. Make a '
                         'ranged spell attack against the target. On a hit, '
                         'the target takes 4d4 acid damage immediately and 2d4 '
                         'acid damage at the end of its next turn. On a miss, '
                         'the arrow splashes the target with acid for half as '
                         'much of the initial damage and no damage at the end '
                         'of its next turn.'],
                'duration': 'Instantaneous',
                'higher_level': ['When you cast this spell using a spell slot '
                                 'of 3rd level or higher, the 

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/races"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_races = {}
race_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('subraces') :
         race_data['subraces'] = [item['name'] for item in resp2['subraces']]

    if resp2.get('starting_proficiencies'):
        race_data['starting_proficiencies'] = resp2['starting_proficiencies']

    bonus_dict = {}
    for item in resp2['ability_bonuses']:
        bonus_dict[item['ability_score']['name']] = item['bonus']
        
    race_data = {
        'name' : name,
        'speed': resp2['speed'],
        'ability_bonuses': bonus_dict,
        'alignment': resp2['alignment'],
        'age': resp2['age'],
        'size': resp2['size'],
        'size_description': resp2['size_description'],
        'languages': [item['name'] for item in resp2['languages']],
        'language_description': resp2['language_desc'],
        'traits': [item['name'] for item in resp2['traits']],
    }
    dict_of_races[entry['index']] = race_data

pprint.pprint(dict_of_races)

{'dragonborn': {'ability_bonuses': {'CHA': 1, 'STR': 2},
                'age': 'Young dragonborn grow quickly. They walk hours after '
                       'hatching, attain the size and development of a '
                       '10-year-old human child by the age of 3, and reach '
                       'adulthood by 15. They live to be around 80.',
                'alignment': 'Dragonborn tend to extremes, making a conscious '
                             'choice for one side or the other in the cosmic '
                             'war between good and evil. Most dragonborn are '
                             'good, but those who side with evil can be '
                             'terrible villains.',
                'language_description': 'You can speak, read, and write Common '
                                        'and Draconic. Draconic is thought to '
                                        'be one of the oldest languages and is '
                                       

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/subraces"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_subraces = {}
subraces_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('starting_proficiencies'):
        subraces_data['starting_proficiencies'] = [item['name'] for item in resp2['starting_proficiencies']]

    if resp2.get('languages'):
        subraces_data['languages'] = resp2['languages']

    if resp2.get('language_options'):
        languages = resp2['language_options']['from']['options']
        language_names = [lang['item']['name'] for lang in languages]
        subraces_data['language_options'] = language_names

    bonus_dict = {}
    for item in resp2['ability_bonuses']:
        bonus_dict[item['ability_score']['name']] = item['bonus'] 

    subraces_data = {
        'name' : name,
        'description': "".join(resp2["desc"]),
        'race': resp2['race']['name'],
        'ability_bonuses': bonus_dict,
        'racial_traits': [item['name'] for item in resp2['racial_traits']],
    }

    dict_of_subraces[entry['index']] = subraces_data
    

pprint.pprint(dict_of_subraces)

{'high-elf': {'ability_bonuses': {'INT': 1},
              'name': 'High Elf',
              'race': 'Elf',
              'racial_traits': ['Elf Weapon Training',
                                'High Elf Cantrip',
                                'Extra Language']},
 'hill-dwarf': {'ability_bonuses': {'WIS': 1},
                'name': 'Hill Dwarf',
                'race': 'Dwarf',
                'racial_traits': ['Dwarven Toughness']},
 'language_options': ['Dwarvish',
                      'Giant',
                      'Gnomish',
                      'Goblin',
                      'Halfling',
                      'Orc',
                      'Abyssal',
                      'Celestial',
                      'Draconic',
                      'Deep Speech',
                      'Infernal',
                      'Primordial',
                      'Sylvan',
                      'Undercommon'],
 'lightfoot-halfling': {'ability_bonuses': {'CHA': 1},
                        'name':

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/proficiencies"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_proficiencies = {}
proficiency_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('classes'):
         proficiency_data['classes'] = [item['name'] for item in resp2['classes']]

    if resp2.get('races'):
        proficiency_data['races'] = [item['name'] for item in resp2['races']]

    proficiency_data = {
        'name' : name,
        'type' : resp2['type'],
    }
    dict_of_proficiencies[entry['index']] = proficiency_data
pprint.pprint(dict_of_proficiencies)


{'alchemists-supplies': {'name': "Alchemist's Supplies",
                         'type': "Artisan's Tools"},
 'all-armor': {'name': 'All armor', 'type': 'Armor'},
 'bagpipes': {'name': 'Bagpipes', 'type': 'Musical Instruments'},
 'battleaxes': {'name': 'Battleaxes', 'type': 'Weapons'},
 'blowguns': {'name': 'Blowguns', 'type': 'Weapons'},
 'breastplate': {'name': 'Breastplate', 'type': 'Armor'},
 'brewers-supplies': {'name': "Brewer's Supplies", 'type': "Artisan's Tools"},
 'calligraphers-supplies': {'name': "Calligrapher's Supplies",
                            'type': "Artisan's Tools"},
 'carpenters-tools': {'name': "Carpenter's Tools", 'type': "Artisan's Tools"},
 'cartographers-tools': {'name': "Cartographer's Tools",
                         'type': "Artisan's Tools"},
 'chain-mail': {'name': 'Chain Mail', 'type': 'Armor'},
 'chain-shirt': {'name': 'Chain Shirt', 'type': 'Armor'},
 'classes': ['Rogue'],
 'clubs': {'name': 'Clubs', 'type': 'Weapons'},
 'cobblers-tools': {'name': 

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/equipment"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()
dict_of_equipment = {}
equipment_data = {}

for entry in resp['results']:
    name_of_equip = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('contents'):
       equipment_data['contents'] = [item['name'] for item in resp2['contents']]

    if resp2.get('properties'):
         equipment_data['properties'] = [item['name'] for item in resp2['properties']]

    if resp2.get('contents'):
        equipment_data['contents'] = [{'name': item['item']['name']} for item in resp2['contents']
    ]

    equipment_data =  {
        'name' : name_of_equip,
        'desc' : "".join(resp2["desc"]),
        'special' : resp2['special'],
        'equipment-category': resp2['equipment_category']['name'],
        'gear-category': resp2.get('gear_category',{}).get('name')
    }
    dict_of_equipment[entry['index']] = equipment_data
pprint.pprint(dict_of_equipment)


{'abacus': {'contents': [],
            'desc': [],
            'equipment-category': 'Adventuring Gear',
            'gear-category': 'Standard Gear',
            'name': 'Abacus',
            'special': []},
 'acid-vial': {'contents': [],
               'desc': ['As an action, you can splash the contents of this '
                        'vial onto a creature within 5 feet of you or throw '
                        'the vial up to 20 feet, shattering it on impact. In '
                        'either case, make a ranged attack against a creature '
                        'or object, treating the acid as an improvised weapon.',
                        'On a hit, the target takes 2d6 acid damage.'],
               'equipment-category': 'Adventuring Gear',
               'gear-category': 'Standard Gear',
               'name': 'Acid (vial)',
               'special': []},
 'alchemists-fire-flask': {'contents': [],
                           'desc': ['This sticky, adhesive fluid ignites w

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/features"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_features = {}
feature_data = {}

for entry in resp['results']:
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('prerequisites'):
        feature_data['prerequisites'] =  resp2['prerequisites']
    
    feature_data = {
        'name' : entry['name'],
        'desc' : "".join(resp2["desc"]),
        'class': resp2['class']['name'],
        'level': resp2['level']
    }
    dict_of_features[entry['index']] = feature_data
pprint.pprint(dict_of_features)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/magic-items"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_magic_items = {}
item_data = {}

for entry in resp['results']:
    name_of_item = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('variants'):
         item_data['variants'] = [item['name'] for item in resp2['variants']]

    item_data = {
        'name' : entry['name'],
        'desc' : "".join(resp2["desc"]),
        'equipment-category': resp2['equipment_category']['name'],
        'rarity': resp2['rarity']['name'],
    }
    dict_of_magic_items[entry['index']] = item_data
pprint.pprint(dict_of_magic_items)


{'adamantine-armor': {'desc': 'Armor (medium or heavy, but not hide), '
                              'uncommonThis suit of armor is reinforced with '
                              'adamantine, one of the hardest substances in '
                              "existence. While you're wearing it, any "
                              'critical hit against you becomes a normal hit.',
                      'equipment-category': 'Armor',
                      'name': 'Adamantine Armor',
                      'rarity': 'Uncommon',
                      'variants': [{'index': 'ammunition-1',
                                    'name': 'Ammunition, +1',
                                    'url': '/api/2014/magic-items/ammunition-1'},
                                   {'index': 'ammunition-2',
                                    'name': 'Ammunition, +2',
                                    'url': '/api/2014/magic-items/ammunition-2'},
                                   {'index': 'ammunition-3',


In [ ]:
url = "https://www.dnd5eapi.co/api/2014/monsters"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_monsters = {}
monster_data = {}

for entry in resp['results']:
    name_of_monster = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('armor_class'):
       armor_class = {}
       for item in resp2['armor_class']:
            armor_class[item['type']] = item['value']
       monster_data['armor_class'] = armor_class

    if resp2.get('damage_vulnerabilities'):
        monster_data['damage_vulnerabilites'] = resp2['damage_vulnerabilities']

    if resp2.get('damage_resistances'):
        monster_data['damage_resistances'] = resp2['damage_resistances']
    
    if resp2.get('damage_immunities'):
        monster_data['damage_immunities'] = resp2['damage_immunities']

    if resp2.get('condition_immunities'):
        monster_data['condition_immunities'] = [item['name'] for item in resp2['condition_immunitites']]

    if resp2.get('special_abilites'):
        special = {}
        for items in resp2['special_abilities']:
            special['name'] = items['name']
            special['description'] = items['desc']
            if 'damage' in items:
                special['damage'] = items['damage']
            if 'dc' in items:
                dc = {}
                dc['name'] = items['dc']['dc_type']['name']
                dc['value'] = items['dc']['dc_value']
                special['dc'] = dc
        monster_data['special_abilities'] = special
    
    if resp2.get('actions'):
        for items in resp2['actions']:
            actions = {}
            actions['name'] = items['name']
            actions['description'] = items['desc']
        monster_data['actions'] = actions

    if resp2.get('legendary_actions'):
        legendary = {}
        for items in resp2['legendary_actions']:
            legendary['name'] = items['name']
            legendary['action_description'] = items['desc']
        monster_data['legendary_actions'] = legendary

    if resp2.get('forms'):
        monster_data['forms'] = [item['name'] for item in resp2['forms']]

    if resp2.get('reactions'):
        reactions = {}
        for item in resp2['reactions']:
             reactions['name'] = item['name']
             reactions['description'] = item['desc']
        monster_data['reactions'] = reactions

    senses = {}
    for key, value in resp2['senses'].items():
        senses[key] = value
    
    movement = {}
    for key, value in resp2['speed'].items():
        movement[key] = value
    
    proficiency_monster = {}
    if resp2.get('proficiencies'):
        for items in resp2['proficiencies']:
            proficiency_monster[items['proficiency']['name']] = items['value']
        monster_data['proficiencies'] = proficiency_monster

    monster_data = {
        'name' : name_of_monster,
        'size': resp2['size'],
        'type':resp2['type'],
        'alignment': resp2['alignment'],
        'hit_points':resp2['hit_points'],
        'hit_dice':resp2['hit_dice'],
        'hit_points_roll': resp2['hit_points_roll'],
        'speed': movement,
        'strength': resp2['strength'],
        'dexterity':resp2['dexterity'],
        'constitution': resp2['constitution'],
        'intelligence': resp2['intelligence'],
        'wisdom': resp2['wisdom'],
        'charisma': resp2['charisma'],
        'senses': senses,
        'languages': resp2['languages'],
        'challenge_rating': resp2['challenge_rating'],
        'proficiency_bonus': resp2['proficiency_bonus'],
        'gained_experience': resp2['xp']
    }
    dict_of_monsters[entry['index']] = monster_data
pprint.pprint(dict_of_monsters)



In [ ]:
url = "https://www.dnd5eapi.co/api/2014/equipment-categories"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_equipment_categories = {}
category_data = {}

for entry in resp['results']:
    name = entry['name']
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    if resp2.get('equipment'):
        category_data['type'] = [item['name'] for item in resp2['equipment']]

    category_data = {
        'name' : name
    }
    dict_of_equipment_categories[entry['index']] = category_data
    

pprint.pprint(dict_of_equipment_categories)


{'adventuring-gear': {'name': 'Adventuring Gear',
                      'races': ['Arrow',
                                'Blowgun needle',
                                'Crossbow bolt',
                                'Sling bullet',
                                'Ammunition, +1, +2, or +3',
                                'Arrow of Slaying',
                                'Ammunition, +1',
                                'Ammunition, +2',
                                'Ammunition, +3']},
 'ammunition': {'name': 'Ammunition',
                'races': ['Crystal', 'Orb', 'Rod', 'Staff', 'Wand']},
 'arcane-foci': {'name': 'Arcane Foci',
                 'races': ['Padded Armor',
                           'Leather Armor',
                           'Studded Leather Armor',
                           'Hide Armor',
                           'Chain Shirt',
                           'Scale Mail',
                           'Breastplate',
                           'Half Plate Armor